In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# MULTIPLE ASSETS FRONTIERS AND CAL WITH LEVERAGE (SUM CAN DIFFER FROM 1)
# USING LAGRANGE MULTIPLIERS (ONLY WORKS FOR UNLIMITED SHORT SELLING). ACTUALLY I FEEL THAT THIS APPROACH IS MORE INTUITIVELY IN TERMS OF CODING

# 1. When we only need the efficient frontier (Markowitz frontier)

# Calculate the Markowitz portfolios from a given targeted returns => define the targeted_returns matrix first
def ef_unlimited_calculation(n_assets, return_matrix, covar_matrix, targeted_returns):
    """
    Calculate the standard deviation (risk) of the portfolio for a range of target returns using Lagrange multipliers.
    Allows for unlimited short selling.
    Returns a DataFrame with standard deviation (risk) and targeted returns, and the minimum-variance portfolio (std_mv, r_mv).
    """
    e = np.ones((n_assets, 1))  # 1-vector as a column vector
    return_matrix = return_matrix.reshape((n_assets, 1))  # Ensure return_matrix is a column vector
    
    # Alpha, delta, and zeta calculations
    alpha = (return_matrix.T @ np.linalg.inv(covar_matrix) @ e).item()  # Scalar
    delta = (e.T @ np.linalg.inv(covar_matrix) @ e).item()  # Scalar
    zeta = (return_matrix.T @ np.linalg.inv(covar_matrix) @ return_matrix).item()  # Scalar
    
    r_mv = alpha / delta  # Expected return of the minimum variance portfolio
    
    # Standard deviation of the minimum variance portfolio
    std_mv = np.sqrt(1 / delta)

    # Lists to store results for the efficient frontier
    std_devs = []
    targeted_returns_list = []

    # Calculate portfolio variance and standard deviation for each target return
    for targeted_return in targeted_returns:
        var_portfolio = (1 / delta) + ((delta / (zeta * delta - alpha**2)) * ((targeted_return - r_mv)**2))
        std_dev = np.sqrt(var_portfolio)
        std_devs.append(std_dev)
        targeted_returns_list.append(targeted_return)
    
    # Create a DataFrame with targeted returns and corresponding risks
    df = pd.DataFrame({
        'Targeted Return': targeted_returns_list,
        'Standard Deviation': std_devs })
    
    return df, std_mv, r_mv  # Return DataFrame and minimum-variance portfolio (std_mv, r_mv)

def ef_unlimited_plot(df, std_mv, r_mv, mode = 'separate'):
    """
    Plot the efficient frontier using the DataFrame containing standard deviations and targeted returns.
    Mark the Minimum-Variance Portfolio on the plot.
    Display the region below the MVP as a dashed grey line if mode='separate'.
    
    Parameters:
    df (DataFrame): DataFrame containing 'Standard Deviation' and 'Targeted Return'.
    std_mv (float): Standard deviation of the minimum-variance portfolio.
    r_mv (float): Targeted return of the minimum-variance portfolio.
    mode (str): 'separate' to split the line into efficient and inefficient regions. 
                Any other value will plot the full frontier as a solid line.
    """
    plt.figure(figsize=(15, 7))

    if mode == 'separate':
        # Separate the data points below and above the minimum-variance portfolio
        below_mvp = df[df['Targeted Return'] < r_mv]
        above_mvp = df[df['Targeted Return'] >= r_mv]

        # Plot the part below the MVP as a dashed grey line
        if not below_mvp.empty:
            plt.plot(below_mvp['Standard Deviation'], below_mvp['Targeted Return'], linestyle='--', color='grey', linewidth=2, label="Inefficient Region")

        # Plot the part above the MVP as a solid blue line
        if not above_mvp.empty:
            plt.plot(above_mvp['Standard Deviation'], above_mvp['Targeted Return'], 'b-', label="Efficient Frontier", linewidth=2)
    else:
        # Plot the entire efficient frontier as a solid line
        plt.plot(df['Standard Deviation'], df['Targeted Return'], 'b-', label="Efficient Frontier", linewidth=2)

    # Plot the Minimum-Variance Portfolio
    plt.scatter(std_mv, r_mv, color='red', label='Minimum-Variance Portfolio', s=100, zorder=5)
    
    plt.xlabel("Standard Deviation", fontsize=14, color = 'navy')
    plt.ylabel("Return", fontsize=14, color = 'navy')
    # plt.yticks(np.arange(0.00 ,0.021, 0.0005)) # OPTIONAL
    plt.title("Efficient Frontier", fontsize=16, color='red')
    plt.grid(True) # OPTIONAL
    plt.legend(loc='best')
    plt.show()

# # Example
# df = ef_unlimited_calculation(n_assets, return_matrix, covar_matrix, targeted_returns) # calculate Markowitz portfolio
# ef_unlimited_plot(df[0], df[1], df[2], mode = 'separate') # std_mv and r_mv is generated in ef_unlimited_calculation

In [6]:
# 2. When we need the Capital Allocation Line (CAL)
# To incorporate the risk-free asset into your current model, you can plot the Capital Allocation Line (CAL), which shows the risk-return tradeoff between a risk-free asset and the optimal risky portfolio (in your case, this will be the portfolio with the highest Sharpe ratio on the efficient frontier). The slope of the CAL is the Sharpe ratio of the optimal risky portfolio, and it starts at the risk-free rate on the y-axis.

def ef_unlimited_cal_plot(df, std_mv, r_mv, risk_free_rate, mode='separate'):
    """
    Plot the efficient frontier along with the Capital Allocation Line (CAL).
    Mark the Minimum-Variance Portfolio and the Tangency Portfolio (Max Sharpe Ratio Portfolio).

    Parameters:
    df (DataFrame): DataFrame containing 'Standard Deviation' and 'Targeted Return'.
    std_mv (float): Standard deviation of the minimum-variance portfolio.
    r_mv (float): Targeted return of the minimum-variance portfolio.
    risk_free_rate (float): The risk-free rate used to calculate the CAL line.
    mode (str): 'separate' to split the line into efficient and inefficient regions.
                'single' to plot the full frontier as a solid line.
    """
    plt.figure(figsize=(15, 7))

    # Calculate the Sharpe ratio for each portfolio on the frontier
    df['Sharpe Ratio'] = (df['Targeted Return'] - risk_free_rate) / df['Standard Deviation']
    
    # Identify the maximum Sharpe ratio portfolio (tangency portfolio)
    max_sharpe_idx = df['Sharpe Ratio'].idxmax()
    max_sharpe_portfolio = df.loc[max_sharpe_idx]
    max_sharpe_std = max_sharpe_portfolio['Standard Deviation']
    max_sharpe_return = max_sharpe_portfolio['Targeted Return']

    # Plot the efficient frontier
    if mode == 'separate':
        # Sort data by targeted return (ascending)
        df_sorted = df.sort_values(by='Targeted Return')

        # Separate the data points below and above the minimum-variance portfolio
        below_mvp = df_sorted[df_sorted['Targeted Return'] < r_mv]
        above_mvp = df_sorted[df_sorted['Targeted Return'] >= r_mv]

        # Plot the part below the MVP as a dashed grey line
        if not below_mvp.empty:
            plt.plot(below_mvp['Standard Deviation'], below_mvp['Targeted Return'], linestyle='--', color='grey', linewidth=2, label="Inefficient Region")

        # Plot the part above the MVP as a solid blue line
        if not above_mvp.empty:
            plt.plot(above_mvp['Standard Deviation'], above_mvp['Targeted Return'], 'b-', label="Efficient Frontier", linewidth=2)
    else:
        # Plot the entire efficient frontier as a solid line
        plt.plot(df['Standard Deviation'], df['Targeted Return'], 'b-', label="Efficient Frontier", linewidth=2)

    # Plot the Minimum-Variance Portfolio
    plt.scatter(std_mv, r_mv, color='red', label='Minimum-Variance Portfolio', s=100, zorder=5)

    # Plot the Tangency Portfolio (Max Sharpe Ratio Portfolio)
    plt.scatter(max_sharpe_std, max_sharpe_return, color='green', label='Max Sharpe Ratio Portfolio', s=100, zorder=5)

    # Calculate and plot the CAL line
    cal_x = np.linspace(0, max_sharpe_std*1.5, 100)  # Extend the CAL line beyond the max Sharpe portfolio
    cal_y = risk_free_rate + (max_sharpe_return - risk_free_rate) / max_sharpe_std * cal_x  # CAL equation

    plt.plot(cal_x, cal_y, linestyle='--', color='orange', label='Capital Allocation Line (CAL)', linewidth=2)
    
    plt.xlabel("Standard Deviation", fontsize=14, color =  'navy')
    plt.ylabel("Targeted Return", fontsize=14, color =  'navy')
    plt.title("Efficient Frontier with CAL", fontsize=16, color='red')
    # plt.yticks(np.arange(0.00 ,0.021, 0.0005)) # OPTIONAL
    plt.grid(True) # OPTIONAL
    plt.legend(loc='best')
    plt.show()

In [8]:
# 3. If we need to access the Max Sharpe Ratio portfolio
# NOAH HAS COMPARED THE RESULT BETWEEN THE LAGRANGE APPROACH AND SCIPY OPTIMIZER, THUS JUST KEEP FINE TUNING EVERY THING

def ef_unlimited_msr_components(n_assets, asset_names, return_matrix, covar_matrix, risk_free):

    e = np.ones((n_assets, 1))  # 1-vector as a column vector
    return_matrix = return_matrix.reshape((n_assets, 1))  # Ensuring return_matrix is a column vector
    
    # Alpha, delta, and zeta calculations
    alpha = (return_matrix.T @ np.linalg.inv(covar_matrix) @ e).item()  # Extract scalar
    delta = (e.T @ np.linalg.inv(covar_matrix) @ e).item()  # Extract scalar
    zeta = (return_matrix.T @ np.linalg.inv(covar_matrix) @ return_matrix).item()  # Extract scalar
    r_mv = alpha / delta  # Expected return of the minimum variance portfolio

    # Sharpe ratio related terms
    special_term = zeta - (2 * alpha * risk_free) + (delta * risk_free**2)
    tangency_slope = special_term ** 0.5
    tangency_er = (alpha * risk_free - zeta) / (delta * risk_free - alpha)
    tangency_sd = -tangency_slope / (delta * (risk_free - r_mv))

    # Calculate portfolio weights for the tangency portfolio
    lmbda = (tangency_er - risk_free) / special_term  # Lambda term for weight scaling
    tangency_weights = lmbda * np.linalg.inv(covar_matrix) @ (return_matrix - risk_free * e)  # Tangency portfolio weights
    
    # Calculate total weight of tangency portfolio
    total_tangency_weight = np.sum(tangency_weights)
    
    # Calculate risk-free allocation
    risk_free_allocation = 1 - total_tangency_weight
    
    # Format the output with asset names
    weights_info = "\n".join([f'{name}: {weight:.4f}' for name, weight in zip(asset_names, tangency_weights.flatten())])

    return (f'The weights vector of the tangency portfolio is:\n{weights_info};\n'
            f'The allocation to the risk-free asset is {risk_free_allocation:.4f};\n'
            f'The expected return of the tangency portfolio is {tangency_er:.4f};\n'
            f'The standard deviation of the tangency portfolio is {tangency_sd:.4f};\n'
            f'The slope of the tangency portfolio (max Sharpe ratio) is {tangency_slope:.4f}.')